# Call an API to get some data

We can call any APIs that are in our Azure 'API Management Instance'.
This is basically a gateway to other APIs. We can add any that meet the OpenAPI Specification.

## PreReqs
- Create an 'API Management Instance'
- Add & publish an API
- These APIs can be open or secured
    - If secured you'll need a subscription key from APIM to connect

## Demo

For secured APIs we'll need to save the subscription key as a `DATABASE SCOPED CREDENTIAL`

1. We'll need a database master key with a proper strong password

In [ ]:
IF NOT EXISTS(SELECT * FROM sys.symmetric_keys WHERE name = '##MS_DatabaseMasterKey##')
BEGIN
    create master key encryption by password = '*********'
END

2. Create the `DATABASE SCOPED CREDENTIAL` for the subscription key

In [ ]:
IF NOT EXISTS (SELECT 1 FROM sys.database_credentials WHERE name = 'https://pom-api.azure-api.net/conference/speakers')
BEGIN
    -- create a database scoped credential for managed identity (or Request Headers or Query String)
    CREATE DATABASE SCOPED CREDENTIAL [https://pom-api.azure-api.net/conference/speakers]
    WITH IDENTITY = 'HTTPEndpointHeaders', SECRET = '{"Ocp-Apim-Subscription-Key":"*********"}';
END

3. Call the API and store the response in a temp table

In [ ]:
drop table if exists #temp 

declare @url varchar(500) = 'https://pom-api.azure-api.net/conference/speakers'

declare @ret as int, @response as nvarchar(max);

exec @ret = sp_invoke_external_rest_endpoint
	 @method = 'GET',
	 @url = @url,
     @credential = [https://pom-api.azure-api.net/conference/speakers],
 	 @response = @response output;
	
select @ret as ReturnCode, @response as Response
into #temp

declare @data as nvarchar(max);

select 
	@data = JSON_QUERY(response, '$.result.collection.items')
from #temp;

select 
	ReturnCode, Response
from #temp

4. Parse the JSON response with T-SQL
    - then store it in your database or ...

In [ ]:
declare @data as nvarchar(max);

select 
	@data = JSON_QUERY(response, '$.result.collection.items')
from #temp;

SELECT [value] as FullName, url
FROM OPENJSON(@data) WITH (
	data2 NVARCHAR(MAX) '$.data' AS JSON,
    url NVARCHAR(50) '$.href'
    )CROSS APPLY OPENJSON(data2)
WITH ([value] NVARCHAR(50))